In [ ]:
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Load csv
SLEEP_FITBIT_DATA = pd.read_csv(r'DATASETS\SLEEP\SLEEP_FITBIT_DATA.csv')

In [ ]:
SLEEP_FITBIT_DATA.head

In [ ]:
#Create dictionary of ideal values for each score; calculate L2 norm to generate score)
IDEAL_VALS = {'HOURS_OF_SLEEP': 8, 'REM_SLEEP': 0.225, 'DEEP_SLEEP': 0.18, 'HEART_RATE_BELOW_RESTING': 0.58}
columns = SLEEP_FITBIT_DATA.columns.values.tolist()
columns = columns[:4]
#Generate scores for each lever - compare iteratively and generate average score on its basis
def generate_score(row):
    #Iterate over each column except for gender
    print(" =================================== ")
    total_score = 0
    for column in columns:
        if column == "SCORE": continue
        print(row[column], IDEAL_VALS[column])
        column_score = abs(row[column] - IDEAL_VALS[column])
        print(column_score)
        total_score += column_score
    #Average out the score
    row["SCORE"] = total_score / len(columns)
    print('END SCORE: ', row["SCORE"])
    return row

SLEEP_FITBIT_DATA = SLEEP_FITBIT_DATA.apply(generate_score, axis = "columns")

In [ ]:
print(SLEEP_FITBIT_DATA)

In [ ]:
print(SLEEP_FITBIT_DATA.head)
SLEEP_FITBIT_DATA.to_pickle('PREPROCESSED_SLEEP_FITBIT_DATA.pkl')

In [ ]:
#Reshape tensors for XGBOOST
import tensorflow as tf
from sklearn.model_selection import train_test_split

x_features = SLEEP_FITBIT_DATA.iloc[:, :3]
print(x_features)
y_labels = SLEEP_FITBIT_DATA["SCORE"]
train_x, valid_x, train_y, valid_y = train_test_split(x_features, y_labels, random_state = 2, shuffle = True)

train_x = tf.convert_to_tensor(train_x)
train_x = tf.reshape(train_x, [len(train_x), 3])

valid_x = tf.convert_to_tensor(valid_x)
valid_x = tf.reshape(valid_x, [len(valid_x), 3])

train_y = tf.convert_to_tensor(train_y)
train_y = tf.reshape(train_y, [len(train_y), 1])

valid_y = tf.convert_to_tensor(valid_y)
valid_y = tf.reshape(valid_y, [len(valid_y), 1])

In [ ]:
#TRAIN GRADIENT BOOSTED TREE
from sklearn.metrics import mean_squared_error, mean_absolute_error

#Convert to Dmatrix
TRAIN_DATA = xgb.DMatrix(train_x, train_y, feature_names = columns[1:15])
VALID_DATA = xgb.DMatrix(valid_x, valid_y, feature_names = columns[1:15])
#Parameters for boosted tree
XGBOOST_PARAMS = {"objective": "reg:squarederror", "subsample": 0.6,
                  "colsample_bytree" : 0.6, "learning_rate" : 0.1, "max_depth" : 100,
                  "alpha": 20, "n_estimators": 12}
#Train the xgboost model
XGB_SLEEP_MODEL = xgb.train(XGBOOST_PARAMS, TRAIN_DATA, evals = [(TRAIN_DATA, "TRAIN_DATA"), (VALID_DATA, "VALID_DATA")],
                            num_boost_round = 200, early_stopping_rounds = 40)
predictions = XGB_SLEEP_MODEL.predict(VALID_DATA)
error = mean_squared_error(valid_y, predictions)
print('ERROR: ', error)

In [ ]:
#Save Model
XGB_SLEEP_MODEL.save_model('METRICS_SLEEP_GRADBOOSTED_MODELS[0.06].model')

In [ ]:
#Print desicion tree -> transparency into exactly what the model is doing
ig, ax = plt.subplots(figsize=(30, 30))
xgb.plot_tree(XGB_SLEEP_MODEL, num_trees=10, ax=ax)
plt.show()